In [1]:
import pymongo
import pandas as pd
from pyhive import hive
from urllib.parse import quote
import urllib.parse

In [2]:
client = pymongo.MongoClient("mongodb://localhost:27017/")
db = client["concessionnaire_automobile"] 
collection = db["clients"]  

cursor = collection.find({})

In [3]:
df = pd.DataFrame(list(cursor))

In [4]:
client.close()

In [5]:
# remove all lines with missing values 
print(df.shape[0])
df.replace('', pd.NA, inplace=True) 
df.dropna(inplace=True)
print(df.shape[0])

200000
198793


In [7]:
df['age'] = pd.to_numeric(df['age'], errors='coerce')
df['taux'] = pd.to_numeric(df['taux'], errors='coerce')
df['nbEnfantsAcharge'] = pd.to_numeric(df['nbEnfantsAcharge'], errors='coerce')
df = df[(df['age'] >= 0) & (df['taux'] >= 0) & (df['nbEnfantsAcharge'] >= 0)]
print(df.shape[0])

197641


In [8]:
gender_distinct_values = df['sexe'].unique()
print(gender_distinct_values)

['F' 'M' 'N/D' 'Homme' 'Femme' 'Masculin' '?' 'Feminin']


In [9]:
gender_counts = df['sexe'].value_counts()
print(gender_counts)

sexe
M           135040
F            58203
Masculin      1400
Homme         1335
Femme          639
Feminin        626
N/D            199
?              199
Name: count, dtype: int64


In [10]:
filtered_df = df[~df['sexe'].isin(['N/D', '?']) & (df['sexe']!= '')]
print(filtered_df)
print(filtered_df.shape[0])

                             _id   age sexe    taux situationFamiliale  \
0       663b73d648d9679b84058df4  80.0    F   475.0          En Couple   
1       663b73d648d9679b84058df5  61.0    M   565.0          En Couple   
2       663b73d648d9679b84058df6  56.0    M   400.0          En Couple   
3       663b73d648d9679b84058df7  79.0    M   535.0          En Couple   
4       663b73d648d9679b84058df8  22.0    M   187.0          En Couple   
...                          ...   ...  ...     ...                ...   
199995  663b73f048d9679b84089bfd  63.0    F  1327.0          En Couple   
199996  663b73f048d9679b84089bfe  30.0    F   194.0          En Couple   
199997  663b73f048d9679b84089bff  59.0    M   208.0        Celibataire   
199998  663b73f048d9679b84089c00  75.0    M   161.0          En Couple   
199999  663b73f048d9679b84089c01  21.0    M  1105.0          En Couple   

        nbEnfantsAcharge 2eme voiture immatriculation  
0                    4.0         true      3591 IK 12  

In [11]:
distinct_values = filtered_df['sexe'].unique()
print(distinct_values)

['F' 'M' 'Homme' 'Femme' 'Masculin' 'Feminin']


In [12]:
mapping_gender = {
    'F': 'F',
    'M': 'H',
    'Homme': 'H',
    'Femme': 'F',
    'Masculin': 'H',
    'Feminin': 'F'
}
filtered_df.loc[:, 'sexe'] = filtered_df['sexe'].map(mapping_gender)
print(filtered_df)

                             _id   age sexe    taux situationFamiliale  \
0       663b73d648d9679b84058df4  80.0    F   475.0          En Couple   
1       663b73d648d9679b84058df5  61.0    H   565.0          En Couple   
2       663b73d648d9679b84058df6  56.0    H   400.0          En Couple   
3       663b73d648d9679b84058df7  79.0    H   535.0          En Couple   
4       663b73d648d9679b84058df8  22.0    H   187.0          En Couple   
...                          ...   ...  ...     ...                ...   
199995  663b73f048d9679b84089bfd  63.0    F  1327.0          En Couple   
199996  663b73f048d9679b84089bfe  30.0    F   194.0          En Couple   
199997  663b73f048d9679b84089bff  59.0    H   208.0        Celibataire   
199998  663b73f048d9679b84089c00  75.0    H   161.0          En Couple   
199999  663b73f048d9679b84089c01  21.0    H  1105.0          En Couple   

        nbEnfantsAcharge 2eme voiture immatriculation  
0                    4.0         true      3591 IK 12  

In [13]:
print(filtered_df.shape[0])

197243


In [14]:
ms_counts = filtered_df['situationFamiliale'].value_counts()
print(ms_counts)

situationFamiliale
En Couple      126534
Celibataire     58644
Seule            9662
Marie(e)         1307
Seul              598
?                 202
N/D               186
Divorcee          110
Name: count, dtype: int64


In [15]:
marital_status_distinct_values = filtered_df['situationFamiliale'].unique()
print(marital_status_distinct_values)

['En Couple' 'Celibataire' 'Seule' 'Marie(e)' '?' 'N/D' 'Seul' 'Divorcee']


In [16]:
filtered_df_ = filtered_df[~filtered_df['situationFamiliale'].isin(['N/D', '?']) & (filtered_df['situationFamiliale']!= '')]

In [17]:
ms_counts = filtered_df_['situationFamiliale'].value_counts()
print(ms_counts)

situationFamiliale
En Couple      126534
Celibataire     58644
Seule            9662
Marie(e)         1307
Seul              598
Divorcee          110
Name: count, dtype: int64


In [21]:
filtered_df_copy = filtered_df_.copy()
r_nb_enfants = filtered_df_copy.groupby('situationFamiliale').agg({'nbEnfantsAcharge': 'sum', 'situationFamiliale': 'count'})
r_nb_enfants.rename(columns={'nbEnfantsAcharge': 'sum_nbEnfantsAcharge', 'situationFamiliale': 'count_situationFamiliale'}, inplace=True)
print(r_nb_enfants)

                    sum_nbEnfantsAcharge  count_situationFamiliale
situationFamiliale                                                
Celibataire                          0.0                     58644
Divorcee                           209.0                       110
En Couple                       223834.0                    126534
Marie(e)                          2333.0                      1307
Seul                                 0.0                       598
Seule                            19408.0                      9662


In [27]:
seul_df = filtered_df_copy[filtered_df_copy['situationFamiliale'] == 'Seule']
sum_distinct_nb_enfants_seul = seul_df['nbEnfantsAcharge'].value_counts()
print(sum_distinct_nb_enfants_seul)

nbEnfantsAcharge
3.0    3251
2.0    3244
1.0    3167
Name: count, dtype: int64


In [28]:
mapping_marital_status = {
    'En Couple': 'En Couple',
    'Celibataire': 'Celibataire',
    'Seule': 'Seul(e)',
    'Marie(e)': 'En Couple',
    'Seul': 'Seul(e)',
    'Divorcee': 'Divorce(e)'
}
filtered_df__ = filtered_df_
filtered_df__.loc[:, 'situationFamiliale'] = filtered_df__['situationFamiliale'].map(mapping_marital_status)
print(filtered_df__)

                             _id   age sexe    taux situationFamiliale  \
0       663b73d648d9679b84058df4  80.0    F   475.0          En Couple   
1       663b73d648d9679b84058df5  61.0    H   565.0          En Couple   
2       663b73d648d9679b84058df6  56.0    H   400.0          En Couple   
3       663b73d648d9679b84058df7  79.0    H   535.0          En Couple   
4       663b73d648d9679b84058df8  22.0    H   187.0          En Couple   
...                          ...   ...  ...     ...                ...   
199995  663b73f048d9679b84089bfd  63.0    F  1327.0          En Couple   
199996  663b73f048d9679b84089bfe  30.0    F   194.0          En Couple   
199997  663b73f048d9679b84089bff  59.0    H   208.0        Celibataire   
199998  663b73f048d9679b84089c00  75.0    H   161.0          En Couple   
199999  663b73f048d9679b84089c01  21.0    H  1105.0          En Couple   

        nbEnfantsAcharge 2eme voiture immatriculation  
0                    4.0         true      3591 IK 12  

In [29]:
d = filtered_df__["situationFamiliale"].unique()
print(d)

['En Couple' 'Celibataire' 'Seul(e)' 'Divorce(e)']


In [30]:
_2nd_car_counts = filtered_df__['2eme voiture'].value_counts()
print(_2nd_car_counts)

2eme voiture
false    171096
true      25557
?           202
Name: count, dtype: int64


In [31]:
filtered_df___ = filtered_df__[~filtered_df__['2eme voiture'].isin(['?']) & (filtered_df__['2eme voiture']!= '')]

In [32]:
_2nd_car_counts = filtered_df___['2eme voiture'].value_counts()
print(_2nd_car_counts)

2eme voiture
false    171096
true      25557
Name: count, dtype: int64


In [33]:
# Establish a connection to Hive
conn = hive.Connection(host="localhost", port=10000, username='vagrant' , database = "concessionnaire_automobile")

# Create a cursor object
cursor = conn.cursor()

In [34]:
create_table_sql = """
CREATE TABLE IF NOT EXISTS clients (
    age INT,
    sexe STRING,
    taux DOUBLE,
    situationFamiliale STRING,
    nbEnfantsAcharge INT,
    deuxiemeVoiture STRING,
    immatriculation STRING
)
"""
insert_query = f"INSERT INTO clients (age, sexe, taux, situationFamiliale, nbEnfantsAcharge, deuxiemeVoiture, immatriculation) VALUES (?, ?, ?, ?, ?, ?, ?)"


In [35]:
cursor.execute(create_table_sql)

In [36]:
if '_id' in filtered_df___.columns:
    filtered_df___ = filtered_df___.drop('_id', axis=1)
if "2eme voiture" in filtered_df___.columns:
    filtered_df___.rename(columns={'2eme voiture': 'deuxiemeVoiture'}, inplace=True)
print(filtered_df___.dtypes)

age                   float64
sexe                   object
taux                  float64
situationFamiliale     object
nbEnfantsAcharge      float64
deuxiemeVoiture        object
immatriculation        object
dtype: object


In [37]:
batch_size = 1000

for i in range(0, len(filtered_df___), batch_size):
    batch_df = filtered_df___.iloc[i:i+batch_size]
    values = ','.join(['({})'.format(','.join(["'{}'".format(str(val)) for val in row])) for row in batch_df.values])
    cursor.execute("""
        INSERT INTO clients (age, sexe, taux, situationFamiliale, nbEnfantsAcharge, deuxiemeVoiture, immatriculation)
        VALUES {}
    """.format(values))
    print(i)

0
1000
2000
3000
4000
5000
6000
7000
8000
9000
10000
11000
12000
13000
14000
15000
16000
17000
18000
19000
20000
21000
22000
23000
24000
25000
26000
27000
28000
29000
30000
31000
32000
33000
34000
35000
36000
37000
38000
39000
40000
41000
42000
43000
44000
45000
46000
47000
48000
49000
50000
51000
52000
53000
54000
55000
56000
57000
58000
59000
60000
61000
62000
63000
64000
65000
66000
67000
68000
69000
70000
71000
72000
73000
74000
75000
76000
77000
78000
79000
80000
81000
82000
83000
84000
85000
86000
87000
88000
89000
90000
91000
92000
93000
94000
95000
96000
97000
98000
99000
100000
101000
102000
103000
104000
105000
106000
107000
108000
109000
110000
111000
112000
113000
114000
115000
116000
117000
118000
119000
120000
121000
122000
123000
124000
125000
126000
127000
128000
129000
130000
131000
132000
133000
134000
135000
136000
137000
138000
139000
140000
141000
142000
143000
144000
145000
146000
147000
148000
149000
150000
151000
152000
153000
154000
155000
156000
157000
158000


In [38]:
# Commit the transaction
conn.commit()

# Close cursor and connection
cursor.close()
conn.close()